# Migration between Scotland and overseas by age

Tabs: `SYOA Persons (2001-)` and `SYOA Males (2001-)`

In [1]:
from gssutils import *

if is_interactive():
    scraper = Scraper('https://www.nrscotland.gov.uk/statistics-and-data/statistics/' \
                      'statistics-by-theme/migration/migration-statistics/migration-between-scotland-and-overseas')
#     scraper.run()
    distribution = scraper.distribution(
        mediaType='application/vnd.ms-excel',
        title='Migration between Scotland and overseas by age')
    display(distribution)
    tabs = distribution.as_databaker()

In [2]:
Final_table = pd.DataFrame()

In [3]:
tab = [t for t in tabs if t.name == 'SYOA Males (2001-)'][0]
observations = tab.excel_ref("B6").expand(RIGHT).expand(DOWN).is_not_blank().is_not_whitespace()
age = tab.excel_ref('C5').expand(RIGHT).is_not_blank()
flow = tab.excel_ref('A').expand(DOWN).by_index([3,23,43])
year = tab.excel_ref('A6').expand(DOWN) - tab.excel_ref('A63').expand(DOWN)
Dimensions = [
            HDim(year,'Mid Year',DIRECTLY, LEFT),
            HDim(flow,'Flow',CLOSEST, ABOVE),
            HDim(age,'Age',DIRECTLY, ABOVE),
            HDimConst('Measure Type', 'Count'),
            HDimConst('Unit','People'),
            HDimConst('Sex','M'),
            HDimConst('Area of Destination or Origin','Whole World')
    ]
c1 = ConversionSegment(observations, Dimensions, processTIMEUNIT=True)
new_table = c1.topandas()
new_table['Value'] = pd.to_numeric(new_table['OBS'], errors='coerce').fillna(0)
new_table['Value'] = new_table['Value'].astype(int)
new_table['Flow'] = new_table['Flow'].map(
    lambda x: {
        'In migration of males from overseas 2001-02 to latest' : 'Inflow', 
        'Out migration of males to overseas 2001-02 to latest' : 'Outflow',
        'Net migration of males from overseas 2001-02 to latest': 'Balance' 
        }.get(x, x))
new_table['Age'] = new_table['Age'].astype(str)
new_table['Age'] = new_table['Age'].map(lambda cell:cell.replace('None', 'all'))
new_table['Age'] = 'year/' + new_table['Age']
new_table['Age'] = new_table['Age'].map(lambda cell:cell.replace('.0', ''))
Final_table = pd.concat([Final_table, new_table])

In [4]:
Final_table = Final_table[['Area of Destination or Origin','Mid Year','Sex','Age','Flow','Measure Type','Value','Unit']]

In [5]:
Final_table['Flow'].unique()

array(['Inflow', 'Outflow', 'Balance'], dtype=object)

In [6]:
Final_table.head()

,Area of Destination or Origin,Mid Year,Sex,Age,Flow,Measure Type,Value,Unit
0,Whole World,2001-02,M,year/all,Inflow,Count,14750,People
1,Whole World,2001-02,M,year/0,Inflow,Count,149,People
2,Whole World,2001-02,M,year/1,Inflow,Count,214,People
3,Whole World,2001-02,M,year/2,Inflow,Count,223,People
4,Whole World,2001-02,M,year/3,Inflow,Count,202,People


In [7]:
Final_table.count()

Area of Destination or Origin    4618
Mid Year                         4618
Sex                              4618
Age                              4618
Flow                             4618
Measure Type                     4618
Value                            4618
Unit                             4618
dtype: int64